In [1]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("inverter_data", version="1")
inverter_data_dd = pd.read_parquet(data_asset.path)
inverter_data_dd.head()

Found the config file in: /config.json
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


Resolving access token for scope "https://storage.azure.com/.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


,event_local_time,device_name,metric.AC_CURRENT_A.MEASURED,metric.AC_CURRENT_B.MEASURED,metric.AC_CURRENT_C.MEASURED,metric.AC_CURRENT_MAX.MEASURED,metric.AC_POWER.MEASURED,metric.AC_POWER_LIMIT_SETPOINT.MEASURED,metric.AC_VOLTAGE_AB.MEASURED,metric.AC_VOLTAGE_BC.MEASURED,metric.AC_VOLTAGE_CA.MEASURED,metric.AC_VOLTAGE_HI_SETPOINT.MEASURED,metric.AC_VOLTAGE_LO_SETPOINT.MEASURED,metric.COMM_LINK.MEASURED,metric.DC_BATT_VOLTAGE_BUS.MEASURED,metric.DC_CURRENT.MEASURED,metric.DC_CURRENT_AVG.MEASURED,metric.DC_CURRENT_MAX.MEASURED,metric.DC_POWER.MEASURED,metric.DC_VOLTAGE.MEASURED,metric.DC_VOLTAGE_BUS.MEASURED,metric.DC_VOLTAGE_N.MEASURED,metric.DC_VOLTAGE_P.MEASURED,metric.ENERGY_DELIVERED.MEASURED,metric.ENERGY_DELIVERED_DAILY.MEASURED,metric.ENERGY_DELIVERED_MONTHLY.MEASURED,metric.ENERGY_RECEIVED.MEASURED,metric.FREQUENCY.MEASURED,metric.HEARTBEAT.MEASURED,metric.HW_VERSION.MEASURED,metric.INSUL_MON_AC_RESISTOR.MEASURED,metric.INSUL_MON_DC_RESISTOR.MEASURED,metric.POWER_FACTOR.MEASURED,metric.STATUS_AC_MOD_ADMISSION_TEMP.MEASURED,metric.STATUS_CURRENT_NORMATIVE.MEASURED,metric.STATUS_FAULT_MODULE.MEASURED,metric.STATUS_FAULT_WORD.MEASURED,metric.STATUS_IGBT_MAX_TEMP.MEASURED,metric.STATUS_INSUL_MON_AC.MEASURED,metric.STATUS_INSUL_MON_DC.MEASURED,metric.STATUS_INTERNAL_HUMIDITY.MEASURED,metric.STATUS_INTERNAL_INPUT_WORD.MEASURED,metric.STATUS_INTERNAL_OUTPUT_WORD.MEASURED,metric.STATUS_INTERNAL_TEMP.MEASURED,metric.STATUS_LV_PRESSURE.MEASURED,metric.STATUS_MOD_MAX_TEMP.MEASURED,metric.STATUS_MV_PRESSURE.MEASURED,metric.STATUS_POWER_SOURCE_1.MEASURED,metric.STATUS_POWER_SOURCE_2.MEASURED,metric.STATUS_WARNING_MODULE.MEASURED,metric.STATUS_WARNING_WORD.MEASURED,metric.STATUS_WORD.MEASURED,metric.SVA.MEASURED,metric.SVA_LIMIT_SETPOINT.MEASURED,metric.VAR.MEASURED,metric.VARH_DELIVERED.MEASURED,metric.VARH_DELIVERED_DAILY.MEASURED,metric.VARH_DELIVERED_MONTHLY.MEASURED,metric.VAR_LIMIT_SETPOINT.MEASURED
0,2021-12-02,INV 51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-12-02,INV 52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-02,INV 53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-12-02,INV 54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-12-02,INV 55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Preprocessing

In [3]:
total_power = inverter_data_dd.groupby('event_local_time')['metric.AC_POWER.MEASURED'].sum()
total_power.rename('total_power', inplace=True)
total_power = total_power.reset_index()
total_power['event_local_time'] = pd.to_datetime(total_power['event_local_time'])
total_power.head()

,event_local_time,total_power
0,2021-12-02 00:00:00,0.0
1,2021-12-02 00:05:00,0.0
2,2021-12-02 00:10:00,0.0
3,2021-12-02 00:15:00,0.0
4,2021-12-02 00:20:00,0.0


In [4]:
ac_power_data = inverter_data_dd[['event_local_time', 'device_name', 'metric.AC_POWER.MEASURED']]
ac_power_data.rename(columns={'metric.AC_POWER.MEASURED': 'inverter_ac_power'}, inplace=True)
ac_power_data['inverter_ac_power'].fillna(0, inplace=True)
ac_power_data['event_local_time'] = pd.to_datetime(ac_power_data['event_local_time'])
ac_power_data = pd.merge(
    ac_power_data, 
    total_power, 
    on='event_local_time', 
    how='left'
)
del total_power # Clean up memory

ac_power_data.head()

/tmp/ipykernel_4339/2941808540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ac_power_data.rename(columns={'metric.AC_POWER.MEASURED': 'inverter_ac_power'}, inplace=True)
/tmp/ipykernel_4339/2941808540.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ac_power_data['inverter_ac_power'].fillna(0, inplace=True)
/tmp/ipykernel_4339/2941808540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,event_local_time,device_name,inverter_ac_power,total_power
0,2021-12-02,INV 51,0.0,0.0
1,2021-12-02,INV 52,0.0,0.0
2,2021-12-02,INV 53,0.0,0.0
3,2021-12-02,INV 54,0.0,0.0
4,2021-12-02,INV 55,0.0,0.0


## Identify Failure Sessions

1. **Iterate Over Each Inverter**

   * For every unique `device_name` in the dataset, extract its AC power time-series data.

   * Use `start_time` and `end_time` to track the beginning and end of a failure session.

2. **Logic for Session Tracking**

   1. **Session Start Condition**

      * A failure session **starts** when:

        * `inverter_ac_power == 0`, **and**
        * `total_power > 0` (i.e., the system as a whole is active),
        * and there is currently **no ongoing session** (`start_time is None`).

   2. **Session Continuation**

      * While `inverter_ac_power == 0`, continue updating `end_time` to extend the session duration.

   3. **Session End Condition**

      * When `inverter_ac_power > 0`:

        * If a failure session is in progress (`start_time is not None`), finalize the session:

          * Save `device_name`, `start_time`, and `end_time` as one failure record.
        * Reset both `start_time` and `end_time` to prepare for the next session.

2. **Edge Case Handling**

   * After the final timestamp in the loop:

     * If a session is still open (`start_time` and `end_time` are not `None`), save it to the output.

3. **Result**

   * Each identified session includes the inverter name, the start and end times, and is stored in a DataFrame for further analysis.



In [ ]:
device_names = ac_power_data['device_name'].unique()

failure_sessions = pd.DataFrame(columns=['device_name', 'start_time', 'end_time'])

for inverter in device_names:
    print(f"Inverter: {inverter}")

    inverter_ac_power = ac_power_data[ac_power_data['device_name'] == inverter].set_index('event_local_time')    
    start_time = None
    end_time = None
    for t in inverter_ac_power.index:
        row = inverter_ac_power.loc[t]
        if row['inverter_ac_power'] == 0:
            if start_time is None and row['total_power']>0:
                # Start a new failure session
                start_time = t
            end_time = t
        else:
            if start_time is not None:
                failure_sessions.loc[len(failure_sessions)] = {
                    'device_name': inverter,
                    'start_time': start_time,
                    'end_time': end_time
                }
            # Reset the session
            start_time = None
            end_time = None
    if start_time is not None and end_time is not None:
        failure_sessions.loc[len(failure_sessions)] = {
            'device_name': inverter,
            'start_time': start_time,
            'end_time': end_time
        }

failure_sessions['duration'] = failure_sessions['end_time'] - failure_sessions['start_time']

In [6]:
failure_sessions.to_csv('failure_sessions.csv')

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_asset = Data(
    path='failure_sessions.csv',                     
    type=AssetTypes.URI_FILE,
    name="failure_sessions_data",
    description="DataFrame uploaded from notebook",
    version="1.0"
)

ml_client.data.create_or_update(data_asset)

Uploading failure_sessions.csv (< 1 MB): 0.00B [00:00, ?B/s] (< 1 MB): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.84M/

Data({'path': 'azureml://subscriptions/1649c5cd-f323-4601-b6f1-e6090a8d9313/resourcegroups/capstone/workspaces/test/datastores/workspaceblobstore/paths/LocalUpload/a174a189b9199f732dddf98c9b7622fa52c5b36b469b4fbefedc0b32d29de249/failure_sessions.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'failure_sessions_data', 'description': 'DataFrame uploaded from notebook', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/1649c5cd-f323-4601-b6f1-e6090a8d9313/resourceGroups/capstone/providers/Microsoft.MachineLearningServices/workspaces/test/data/failure_sessions_data/versions/1.0', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/leo9005271/code/Users/leo900527', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x776bac0c54e0>, 'serialize': <msrest.se

## Visualize Failure Sessions

In [ ]:
import sys
sys.path.append('/home/azureuser/cloudfiles/code/Users/leo900527')
from visualize import visualize_failure_sessions

failure_sessions = pd.read_csv('failure_sessions.csv', parse_dates=['start_time', 'end_time'])

failure_sessions['duration'] = pd.to_timedelta(failure_sessions['duration'])
min_failure_duration=pd.Timedelta('3 day')
failure_sessions = failure_sessions[failure_sessions['duration'] >= min_failure_duration]

In [ ]:
html_blocks = []

device_names = ac_power_data['device_name'].unique()
for device_name in device_names:
    print(f"Generating figure for {device_name}")
    fig = visualize_failure_sessions(ac_power_data, failure_sessions, device_name)
    fig.write_html(f"failure_dashboard/{device_name.replace(' ', '_')}.html")
    fig.show()
